# Two step CMR search


First we use the collection search API to find collections of interest.

    GET https://cmr.earthdata.nasa.gov/search/collections

Say, for example, we are interested in surface reflectance data from the Sentinel-2 satellite over New York City on June 26th, 2021.

Let's translate the above into a set of query filters.

## Spatial
New York City is located at 40.70125° North, 73.98581° West. We can use the `point` search filter to express this

    point=-73.98581,40.70125

## Temporal 
June 26th, 2021 can be described using an ISO8601 format as `2021-06-21T00:00:00Z,2021-06-21T23:59:59Z`. We can use the `temporal` search filter to express this

    temporal[]=2021-06-21T00:00:00Z,2021-06-21T23:59:59Z

## Satellite and measurement
Finally, our satellite and measurement can be expressed using a keyword (free text) search filter.

    keyword=sentinel-2 surface reflectance

## The collection search

We can AND all these terms together and form a CMR query as follows:

    GET https://cmr.earthdata.nasa.gov/search/collections?keyword=sentinel-2 surface reflectance&point=-73.98581,40.70125&temporal[]=2021-06-21T00:00:00Z,2021-06-21T23:59:59Z


In [ ]:
import requests
import xml.dom.minidom
import xml.etree.ElementTree as et

response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?keyword=sentinel-2 surface reflectance&point=-73.98581,40.70125&temporal[]=2021-06-21T00:00:00Z,2021-06-21T23:59:59Z")
print("Number of matching collections: " + response.headers['CMR-Hits'])

We have a number of results. Let's look at the first one:

In [ ]:
doc = et.fromstring(response.text)

references = doc.findall('references/reference')
reference = references[0].find('location').text
concept_id = references[0].find('id').text
print("First collection reference: " + str(reference))
print("First collection name: " + str(references[0].find('name').text))
print("First collection concept id: " + str(concept_id))

That looks good. Let's look at the actual metadata associated with that result.
## The collection retrieval

In [ ]:
response = requests.get(reference + ".echo10")
response_as_dom = xml.dom.minidom.parseString(response.text)
xml_reference_response = response_as_dom.toprettyxml()
print(xml_reference_response)

This looks like what we want. What else can we do at the collection level to assess whether we have the right collection?
It has a browse image. Let's look at that. Note: this is a browse image for the whole collection. It's just an example
## Extract the collection browse image and visualize

In [ ]:
doc = et.fromstring(response.text)
browse_url = doc.findall("AssociatedBrowseImageUrls/ProviderBrowseUrl/URL")[0].text
print("Collection browse URL: " + str(browse_url)) 

from PIL import Image

img = Image.open(requests.get(browse_url, stream=True).raw)
img. show()

## Searching for granules
Assuming we are happy with this collection as the source of our data, we can now use the concept-id of that collection to search for granules.
We will use the spatial and temporal filters we used in our collection search. We no longer need the keyword filter

    https://cmr.earthdata.nasa.gov/search/granules.echo10?collection_concept_id=foo&point=-73.98581,40.70125&temporal[]=2021-06-21T00:00:00Z,2021-06-21T23:59:59Z

In [ ]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/granules.echo10?collection_concept_id=" + concept_id + "&point=-73.98581,40.70125&temporal[]=2021-06-21T00:00:00Z,2021-06-21T23:59:59Z")
print("Number of matching granules: " + response.headers['CMR-Hits'])


That search matched a single granule.

## Inspecting the granule metadata
Let's look at the metadata for this granule

In [ ]:
response_as_dom = xml.dom.minidom.parseString(response.text)
xml_reference_response = response_as_dom.toprettyxml()
print(xml_reference_response)


## Extract the granule browse image and visualize
Let's look at the browse for this granule

In [ ]:
doc = et.fromstring(response.text)
browse_url = doc.findall("result/Granule/AssociatedBrowseImageUrls/ProviderBrowseUrl/URL")[0].text
print("Granule browse URL: " + str(browse_url)) 

img = Image.open(requests.get(browse_url, stream=True).raw)
img. show()

New York!

## Where is the data?
We can extract the data locations for this granule as follows:

In [ ]:
data_urls = doc.findall("result/Granule/OnlineAccessURLs/OnlineAccessURL/URL")
for url in data_urls:
    print("Granule data URL: " + url.text)

We have a number of data urls. Generally, Sentinel-2 gives you a data file for each band associated with the instrument.

## Obtaining the data
Coming soon!